# 0. Create helper

In [1]:
import boto3
from botocore.exceptions import BotoCoreError
import json
import requests
from requests.auth import HTTPBasicAuth
from requests_aws4auth import AWS4Auth
import time
from getpass import getpass
import urllib.parse

# This Python code is compatible with AWS OpenSearch versions 2.9 and higher.
class AIConnectorHelper:
    
    def __init__(self,
                 opensearch_domain_region, 
                 opensearch_domain_name,
                 opensearch_domain_username,
                 opensearch_domain_password,
                 aws_user_arn,
                 aws_role_arn,
                 aws_access_key_id,
                 aws_secret_access_key,
                 aws_session_token):
        self.opensearch_domain_region = opensearch_domain_region
        self.opensearch_domain_username = opensearch_domain_username
        self.opensearch_domain_opensearch_domain_password = opensearch_domain_password
        self.aws_user_arn = aws_user_arn
        self.aws_role_arn = aws_role_arn
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.aws_session_token = aws_session_token
        
        # Create the session and clients
        self.session = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_session_token=aws_session_token,
            region_name=opensearch_domain_region
        )
        self.opensearch_client = self.session.client('es')
        self.iam_client = self.session.client('iam')
        self.secretsmanager_client = self.session.client('secretsmanager')
        self.sts_client = self.session.client('sts')
        # Get the OpenSearch domain info
        self.opensearch_domain_url, self.opensearch_domain_arn = self.get_opensearch_domain_info(opensearch_domain_name)
        
    def get_opensearch_domain_info(self, domain_name):
        try:
            response = self.opensearch_client.describe_elasticsearch_domain(DomainName=domain_name)
            domain_info = response['DomainStatus']
            domain_url = domain_info['Endpoint']
            domain_arn = domain_info['ARN']
            return f'https://{domain_url}', domain_arn
        except self.opensearch_client.exceptions.ResourceNotFoundException:
            print(f"Domain '{domain_name}' not found.")
            return None, None
        
    def get_user_arn(self, username):
        if not username:
            return None
        try:
            # Get information about the IAM user
            response = self.iam_client.get_user(UserName=username)
            user_arn = response['User']['Arn']
            return user_arn
        except self.iam_client.exceptions.NoSuchEntityException:
            print(f"IAM user '{username}' not found.")
            return None

    def secret_exists(self, secret_name):
        try:
            self.secretsmanager_client.get_secret_value(SecretId=secret_name)
            return True
        except self.secretsmanager_client.exceptions.ResourceNotFoundException:
            # If a ResourceNotFoundException was raised, the secret does not exist
            return False

    def get_secret_arn(self, secret_name):
        try:
            response = self.secretsmanager_client.describe_secret(SecretId=secret_name)
            return response['ARN']
        except self.secretsmanager_client.exceptions.ResourceNotFoundException:
            print(f"The requested secret {secret_name} was not found")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def create_secret(self, secret_name, secret_value):
        try:
            response = self.secretsmanager_client.create_secret(
                Name=secret_name,
                SecretString=json.dumps(secret_value),
            )
            print(f'Secret {secret_name} created successfully.')
            return response['ARN']  # Return the ARN of the created secret
        except BotoCoreError as e:
            print(f'Error creating secret: {e}')
            return None

    def role_exists(self, role_name):
        try:
            self.iam_client.get_role(RoleName=role_name)
            return True
        except self.iam_client.exceptions.NoSuchEntityException:
            return False

    def create_iam_role(self, role_name, trust_policy_json, inline_policy_json):
        try:
            # Create the role with the trust policy
            create_role_response = self.iam_client.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy_json),
                Description='Role with custom trust and inline policies',
            )

            # Get the ARN of the newly created role
            role_arn = create_role_response['Role']['Arn']

            # Attach the inline policy to the role
            self.iam_client.put_role_policy(
                RoleName=role_name,
                PolicyName='InlinePolicy',  # you can replace this with your preferred policy name
                PolicyDocument=json.dumps(inline_policy_json)
            )

            print(f'Created role: {role_name}')
            return role_arn

        except Exception as e:
            print(f"Error creating the role: {e}")
            return None

    def get_role_arn(self, role_name):
        if not role_name:
            return None
        try:
            response = self.iam_client.get_role(RoleName=role_name)
            # Return ARN of the role
            return response['Role']['Arn']
        except self.iam_client.exceptions.NoSuchEntityException:
            print(f"The requested role {role_name} does not exist")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def map_iam_role_to_backend_role(self, role_arn, os_security_role='ml_full_access'):
        url = f'{self.opensearch_domain_url}/_plugins/_security/api/rolesmapping/{os_security_role}'
        r=requests.get(url, auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password))
        role_mapping = json.loads(r.text)
        headers = {"Content-Type": "application/json"}
        if 'status' in role_mapping and role_mapping['status'] == 'NOT_FOUND':
            data = {'backend_roles': [ role_arn ] }
            response = requests.put(url, headers=headers, data=json.dumps(data), auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password))
            print(response.text)
        else:
            role_mapping = role_mapping[os_security_role]
            role_mapping['backend_roles'].append(role_arn)
            data = [
              {
                "op": "replace", "path": "/backend_roles", "value": list(set(role_mapping['backend_roles']))
              }
            ]
            response = requests.patch(url, headers=headers, data=json.dumps(data), auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password))
            print(response.text)

    def assume_role(self, create_connector_role_arn, role_session_name="your_session_name"):
        assumed_role_object = self.sts_client.assume_role(
            RoleArn=create_connector_role_arn,
            RoleSessionName=role_session_name,
        )

        # Obtain the temporary credentials from the assumed role 
        temp_credentials = assumed_role_object["Credentials"]

        return temp_credentials

    def create_connector(self, create_connector_role_name, payload):
        create_connector_role_arn = self.get_role_arn(create_connector_role_name)
        temp_credentials = self.assume_role(create_connector_role_arn)
        awsauth = AWS4Auth(
            temp_credentials["AccessKeyId"],
            temp_credentials["SecretAccessKey"],
            self.opensearch_domain_region,
            'es',
            session_token=temp_credentials["SessionToken"],
        )

        path = '/_plugins/_ml/connectors/_create'
        url = self.opensearch_domain_url + path

        headers = {"Content-Type": "application/json"}

        r = requests.post(url, auth=awsauth, json=payload, headers=headers)
        print(r.text)
        connector_id = json.loads(r.text)['connector_id']
        return connector_id
    
    def get_task(self, task_id):
        return requests.get(f'{self.opensearch_domain_url}/_plugins/_ml/tasks/{task_id}',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password))
    
    def create_model(self, model_name, description, connector_id, deploy=True):
        payload = {
          "name": model_name,
          "function_name": "remote",
          "description": description,
          "connector_id": connector_id
        }
        headers = {"Content-Type": "application/json"}
        deploy_str = str(deploy).lower()
        r = requests.post(f'{self.opensearch_domain_url}/_plugins/_ml/models/_register?deploy={deploy_str}',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          json=payload,
                          headers=headers)
        print(r.text)
        response = json.loads(r.text)
        if 'model_id' in response:
            return response['model_id']
        else:
            time.sleep(2) # sleep two seconds for task complete
            r = self.get_task(response['task_id'])
            print(r.text)
            return json.loads(r.text)['model_id']
    
    def deploy_model(self, model_id):
        return requests.post(f'{self.opensearch_domain_url}/_plugins/_ml/models/{model_id}/_deploy',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          headers=headers)
    
    def predict(self, model_id, payload):
        headers = {"Content-Type": "application/json"}

        r = requests.post(f'{self.opensearch_domain_url}/_plugins/_ml/models/{model_id}/_predict',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          json=payload,
                          headers=headers)
        return r.text

    def create_index(self, index_name, payload):
        headers = {"Content-Type": "application/json"}

        r = requests.put(f'{self.opensearch_domain_url}/{index_name}',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          json=payload,
                          headers=headers)
        return r.text

    def search(self, index_name, query, search_pipeline=None):
        headers = {"Content-Type": "application/json"}

        search_url = f'{self.opensearch_domain_url}/{index_name}/_search'
        if search_pipeline:
            search_url = f'{self.opensearch_domain_url}/{index_name}/_search?search_pipeline={search_pipeline}'

        r = requests.get(search_url,
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          json=query,
                          headers=headers)
        return r.text

    def bulk_ingest(self, index_name, data):
        # Prepare the bulk request body
        bulk_data = []
        for i, item in enumerate(data, start=1):
            # Add the action line
            bulk_data.append(json.dumps({"index": {"_index": index_name}}))
            # Add the document data
            bulk_data.append(json.dumps(item))
        
        # Join the bulk data with newlines
        bulk_body = "\n".join(bulk_data) + "\n"
        
        # Set the headers
        headers = {"Content-Type": "application/x-ndjson"}
        
        # Send the bulk request with Basic Auth
        response = requests.post(f"{self.opensearch_domain_url}/_bulk",
                                 auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                                 headers=headers,
                                 data=bulk_body)
        
        # Check the response
        if response.status_code == 200:
            print("Bulk insert successful")
            print(json.dumps(response.json(), indent=2))
        else:
            print(f"Error: {response.status_code}")
            print(response.text)

    def create_ingest_pipeline(self, pipeline_name, payload):
        headers = {"Content-Type": "application/json"}

        r = requests.put(f'{self.opensearch_domain_url}/_ingest/pipeline/{pipeline_name}',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          json=payload,
                          headers=headers)
        return r.text
        
    def create_search_pipeline(self, pipeline_name, payload):
        headers = {"Content-Type": "application/json"}

        r = requests.put(f'{self.opensearch_domain_url}/_search/pipeline/{pipeline_name}',
                          auth=HTTPBasicAuth(self.opensearch_domain_username, self.opensearch_domain_opensearch_domain_password),
                          json=payload,
                          headers=headers)
        return r.text
    
    def create_connector_with_secret(self, secret_name, secret_value, connector_role_name, create_connector_role_name, create_connector_input, sleep_time_in_seconds=10):
        # Step1: Create Secret
        print('Step1: Create Secret')
        if not self.secret_exists(secret_name):
            secret_arn = self.create_secret(secret_name, secret_value)
        else:
            print('secret exists, skip creating')
            secret_arn = self.get_secret_arn(secret_name)
        print('----------')
        
        # Step2: Create IAM role configued in connector
        trust_policy = {
          "Version": "2012-10-17",
          "Statement": [
            {
              "Effect": "Allow",
              "Principal": {
                "Service": "es.amazonaws.com"
              },
              "Action": "sts:AssumeRole"
            }
          ]
        }

        inline_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Action": [
                        "secretsmanager:GetSecretValue",
                        "secretsmanager:DescribeSecret"
                    ],
                    "Effect": "Allow",
                    "Resource": secret_arn
                }
            ]
        }

        print('Step2: Create IAM role configued in connector')
        if not self.role_exists(connector_role_name):
            connector_role_arn = self.create_iam_role(connector_role_name, trust_policy, inline_policy)
        else:
            print('role exists, skip creating')
            connector_role_arn = self.get_role_arn(connector_role_name)
        print('----------', connector_role_arn)
        
        # Step 3: Configure IAM role in OpenSearch
        # 3.1 Create IAM role for Signing create connector request
        statements = []
        if self.aws_user_arn:
            statements.append({
                "Effect": "Allow",
                "Principal": {
                    "AWS": self.aws_user_arn
                },
                "Action": "sts:AssumeRole"
            })
        if self.aws_role_arn:
            statements.append({
                "Effect": "Allow",
                "Principal": {
                    "AWS": self.aws_role_arn
                },
                "Action": "sts:AssumeRole"
            })
        trust_policy = {
            "Version": "2012-10-17",
            "Statement": statements
        }

        inline_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": "iam:PassRole",
                    "Resource": connector_role_arn
                },
                {
                    "Effect": "Allow",
                    "Action": "es:ESHttpPost",
                    "Resource": self.opensearch_domain_arn
                }
            ]
        }

        print('Step 3: Configure IAM role in OpenSearch')
        print('Step 3.1: Create IAM role for Signing create connector request')
        if not self.role_exists(create_connector_role_name):
            create_connector_role_arn = self.create_iam_role(create_connector_role_name, trust_policy, inline_policy)
        else:
            print('role exists, skip creating')
            create_connector_role_arn = self.get_role_arn(create_connector_role_name)
        print('----------')
        
        # 3.2 Map backend role
        print(f'Step 3.2: Map IAM role {create_connector_role_name} to OpenSearch permission role')
        self.map_iam_role_to_backend_role(create_connector_role_arn)
        print('----------')
        
        # 4. Create connector
        print('Step 4: Create connector in OpenSearch')
        # When you create an IAM role, it can take some time for the changes to propagate across AWS systems.
        # During this time, some services might not immediately recognize the new role or its permissions.
        # So we wait for some time before creating connector.
        # If you see such error: ClientError: An error occurred (AccessDenied) when calling the AssumeRole operation
        # you can rerun this function.
        
        # Wait for some time
        time.sleep(sleep_time_in_seconds)
        payload = create_connector_input
        payload['credential'] = {
            "secretArn": secret_arn,
            "roleArn": connector_role_arn
        }
        connector_id = self.create_connector(create_connector_role_name, payload)
        print('----------')
        return connector_id
    
    def create_connector_with_role(self, connector_role_inline_policy, connector_role_name, create_connector_role_name, create_connector_input, sleep_time_in_seconds=10):
        # Step1: Create IAM role configued in connector
        trust_policy = {
          "Version": "2012-10-17",
          "Statement": [
            {
              "Effect": "Allow",
              "Principal": {
                "Service": "es.amazonaws.com"
              },
              "Action": "sts:AssumeRole"
            }
          ]
        }

        print('Step1: Create IAM role configued in connector')
        if not self.role_exists(connector_role_name):
            connector_role_arn = self.create_iam_role(connector_role_name, trust_policy, connector_role_inline_policy)
        else:
            print('role exists, skip creating')
            connector_role_arn = self.get_role_arn(connector_role_name)
        print('----------')

        # Step 2: Configure IAM role in OpenSearch
        # 2.1 Create IAM role for Signing create connector request
        statements = []
        if self.aws_user_arn:
            statements.append({
                "Effect": "Allow",
                "Principal": {
                    "AWS": self.aws_user_arn
                },
                "Action": "sts:AssumeRole"
            })
        if self.aws_role_arn:
            statements.append({
                "Effect": "Allow",
                "Principal": {
                    "AWS": self.aws_role_arn
                },
                "Action": "sts:AssumeRole"
            })
        trust_policy = {
            "Version": "2012-10-17",
            "Statement": statements
        }

        inline_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": "iam:PassRole",
                    "Resource": connector_role_arn
                },
                {
                    "Effect": "Allow",
                    "Action": "es:ESHttpPost",
                    "Resource": self.opensearch_domain_arn
                }
            ]
        }

        print('Step 2: Configure IAM role in OpenSearch')
        print('Step 2.1: Create IAM role for Signing create connector request')
        if not self.role_exists(create_connector_role_name):
            create_connector_role_arn = self.create_iam_role(create_connector_role_name, trust_policy, inline_policy)
        else:
            print('role exists, skip creating')
            create_connector_role_arn = self.get_role_arn(create_connector_role_name)
        print('----------')

        # 2.2 Map backend role
        print(f'Step 2.2: Map IAM role {create_connector_role_name} to OpenSearch permission role')
        self.map_iam_role_to_backend_role(create_connector_role_arn)
        print('----------')

        # 3. Create connector
        print('Step 3: Create connector in OpenSearch')
        # When you create an IAM role, it can take some time for the changes to propagate across AWS systems.
        # During this time, some services might not immediately recognize the new role or its permissions.
        # So we wait for some time before creating connector.
        # If you see such error: ClientError: An error occurred (AccessDenied) when calling the AssumeRole operation
        # you can rerun this function.

        # Wait for some time
        time.sleep(sleep_time_in_seconds)
        payload = create_connector_input
        payload['credential'] = {
            "roleArn": connector_role_arn
        }
        connector_id = self.create_connector(create_connector_role_name, payload)
        print('----------')
        return connector_id

In [2]:
def pretty_print_json(json_string):
    # Parse the JSON string
    parsed = json.loads(json_string)
    
    # Pretty print with indentation
    pretty_json = json.dumps(parsed, indent=4, sort_keys=True)
    
    print(pretty_json)

# 1. Set up configurations

## 1.1 OpenSearch configuration

In [3]:
opensearch_domain_region = input('Enter your AWS OpenSearch region, default us-east-1') or 'us-east-1'
opensearch_domain_name = getpass('Enter your AWS OpenSearch domain name') or None
opensearch_domain_username = getpass('Enter your AWS OpenSearch domain admin username') or None
opensearch_domain_password = getpass('Enter your AWS OpenSearch domain password') or None

Enter your AWS OpenSearch region, default us-east-1 
Enter your AWS OpenSearch domain name ········
Enter your AWS OpenSearch domain admin username ········
Enter your AWS OpenSearch domain password ········


## 1.2 AWS Credential configuration

**You must set either `aws_user_arn` or `aws_role_arn`.**

### 1. `aws_user_arn` :

Use AWS IAM user ARN.
To avoid permission issue and quick start, you can use user with `AdministratorAccess` policy.

### 2. `aws_role_arn`

Use AWS IAM role ARN.

If you use AWS role, you can use this AWS CLI to get AWS role temporary credential

`aws sts assume-role --role-arn <aws_role_arn> --role-session-name <your_session_name>`

This AWS role must have such least permission:
1. Required Trust Relationship Policy:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "es.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

Note: Additional trust relationship configuration is needed as you need to use other entities to assume this role for temporary credentials.
For example, if you want an IAM user to assume this role, add the following statement to the trust relationship policy:
```
{
    "Effect": "Allow",
    "Principal": {
        "AWS": "arn:aws:iam::<your_AWS_Account_ID>:user/<your_AWS_IAM_User_Name>"
    },
    "Action": "sts:AssumeRole"
}
```
2. Required Permission Policy:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "es:DescribeElasticsearchDomain",
                "es:DescribeDomain"
            ],
            "Resource": "<your_OpenSearch_Domain_ARN>"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:CreateRole",
                "iam:GetRole",
                "iam:PutRolePolicy",
                "iam:AttachRolePolicy",
                "iam:TagRole",
                "iam:PassRole"
            ],
            "Resource": "arn:aws:iam::<your_AWS_Account_ID>:role/<connector_role_prefix>*"
        }
    ]
}
```
If you need to store API key in AWS SecretManger, you need to add following statement to the permission policy:
```
{
    "Effect": "Allow",
    "Action": [
        "secretsmanager:CreateSecret",
        "secretsmanager:GetSecretValue",
        "secretsmanager:DescribeSecret"
    ],
    "Resource": "arn:aws:secretsmanager:<your_AWS_Region>:<your_AWS_Account_ID>:secret:<secret_prefix>*"
}
```

In [4]:
connector_role_prefix = input('Enter your connector role prefix') or None 
if not connector_role_prefix:
    raise ValueError("You must provide connector_role_prefix")

Enter your connector role prefix my_test_role


In [5]:
secret_prefix = input('Enter your secret prefix') or None 
if not connector_role_prefix:
    raise ValueError("You must provide secret_prefix")

Enter your secret prefix my_test_secret


In [6]:
aws_user_arn = getpass('Enter your AWS IAM user RN') or None
aws_role_arn = getpass('Enter your AWS IAM role ARN') or None 
# You must set either aws_user_arn or aws_role_arn. 
if not bool(aws_user_arn) ^ bool(aws_role_arn):  # XOR operation
    raise ValueError("You must provide exactly one of aws_user_arn or aws_role_arn")

Enter your AWS IAM user RN ········
Enter your AWS IAM role ARN ········


In [7]:
aws_access_key_id = getpass("Enter your AWS Access Key ID: ")
aws_secret_access_key = getpass("Enter your AWS Secret Access Key: ")
aws_session_token = getpass("Enter your AWS Session Token: ")

Enter your AWS Access Key ID:  ········
Enter your AWS Secret Access Key:  ········
Enter your AWS Session Token:  ········


## 1.3 Initialize Connector Helper

In [8]:
helper = AIConnectorHelper(opensearch_domain_region, 
                           opensearch_domain_name, 
                           opensearch_domain_username, 
                           opensearch_domain_password, 
                           aws_user_arn,
                           aws_role_arn,
                           aws_access_key_id,
                           aws_secret_access_key,
                           aws_session_token
                          )

## 2. Create Connector and Model

This demo notebook provides three options
- 2.1 Use DeepSeek Chat service API
- 2.2 Use DeepSeek R1 model on Bedrock
- 2.3 Use DeepSeek R1 model on SageMaker

## 2.1 Use DeepSeek Chat service API

Refer to this [tutorial](https://github.com/opensearch-project/ml-commons/blob/main/docs/tutorials/aws/RAG_with_DeepSeek_Chat_model.md) for more details.

In [9]:
deepseek_api_key = getpass("Enter your DeepSeek API Key: ")

Enter your DeepSeek API Key:  ········


In [10]:
secret_name = f'{secret_prefix}_deepseek_api_secret'
secret_key = f'{secret_prefix}_deepseek_api_key'
secret_value = { secret_key : deepseek_api_key }

In [11]:
# You can use existing role if the role permission and trust relationship are correct.
# But highly suggest to specify new role names. AIConnectorHelper will create role automatically with correct permission.
# If you see permission issue, always try to create new roles.
connector_role_name = f'{connector_role_prefix}_deepseek_chat_model'
create_connector_role_name = f'{connector_role_prefix}_deepseek_chat_model_create'

create_connector_input = {
  "name": "DeepSeek Chat",
  "description": "Test connector for DeepSeek Chat",
  "version": "1",
  "protocol": "http",
  "parameters": {
    "endpoint": "api.deepseek.com",
    "model": "deepseek-chat"
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://${parameters.endpoint}/v1/chat/completions",
      "headers": {
        "Content-Type": "application/json",
        "Authorization": f"Bearer ${{credential.secretArn.{secret_key}}}"
      },
      "request_body": "{ \"model\": \"${parameters.model}\", \"messages\": ${parameters.messages} }"
    }
  ]
}

deepseek_connector_id = helper.create_connector_with_secret(secret_name,
                                                   secret_value, 
                                                   connector_role_name, 
                                                   create_connector_role_name, 
                                                   create_connector_input,
                                                   sleep_time_in_seconds=10)

Step1: Create Secret
Secret my_test_secret_deepseek_api_secret created successfully.
----------
Step2: Create IAM role configued in connector
Created role: my_test_role_deepseek_chat_model
---------- arn:aws:iam::419213735998:role/my_test_role_deepseek_chat_model
Step 3: Configure IAM role in OpenSearch
Step 3.1: Create IAM role for Signing create connector request
Created role: my_test_role_deepseek_chat_model_create
----------
Step 3.2: Map IAM role my_test_role_deepseek_chat_model_create to OpenSearch permission role
{"status":"OK","message":"'ml_full_access' updated."}
----------
Step 4: Create connector in OpenSearch
{"connector_id":"qeRFvpQBFSAM-WczLLLu"}
----------


In [12]:
model_name='DeepSeek Chat Model'
description='DeepSeek Chat Model'
deepseek_model_id = helper.create_model(model_name, description, deepseek_connector_id)
deepseek_model_id

{"task_id":"kylFvpQBts7fa6byLx2Q","status":"CREATED","model_id":"lClFvpQBts7fa6byLx2p"}


'lClFvpQBts7fa6byLx2p'

In [13]:
request_data = {
  "parameters": {
    "messages": [
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello!"
      }
    ]
  }
}
response = helper.predict(deepseek_model_id, request_data)
pretty_print_json(response)

{
    "inference_results": [
        {
            "output": [
                {
                    "dataAsMap": {
                        "choices": [
                            {
                                "finish_reason": "stop",
                                "index": 0.0,
                                "message": {
                                    "content": "Hello! How can I assist you today? \ud83d\ude0a",
                                    "role": "assistant"
                                }
                            }
                        ],
                        "created": 1738359001.0,
                        "id": "2ba6a5be-905e-4814-9460-7c4e8035f029",
                        "model": "deepseek-chat",
                        "object": "chat.completion",
                        "system_fingerprint": "fp_3a5770e1b4",
                        "usage": {
                            "completion_tokens": 11.0,
                            "prompt_cache_hit_tok

## 2.2 Use DeepSeek R1 model on Bedrock

Refer to this [tutorial](https://github.com/opensearch-project/ml-commons/blob/main/docs/tutorials/aws/RAG_with_DeepSeek_R1_model_on_Bedrock.md) for more details.

Make sure you already have DeepSeek R1 model running on Bedrock before running next step.

In [14]:
deepseek_model_bedrock_arn = getpass("Enter your DeepSeek Model Bedrock ARN: ")

Enter your DeepSeek Model Bedrock ARN:  ········


In [15]:
# You can use existing role if the role permission and trust relationship are correct.
# But highly suggest to specify new role names. AIConnectorHelper will create role automatically with correct permission.
# If you see permission issue, always try to create new roles.
connector_role_name = f'{connector_role_prefix}_deepseek_r1_bedrock'
create_connector_role_name = f'{connector_role_prefix}_deepseek_r1_bedrock_create'

bedrock_region = 'us-east-1' # bedrock region could be different with OpenSearch domain region

connector_role_inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Effect": "Allow",
            "Resource": deepseek_model_bedrock_arn
        }
    ]
}
create_connector_input = {
  "name": "DeepSeek R1 model connector",
  "description": "Connector for my Bedrock DeepSeek model",
  "version": "1.0",
  "protocol": "aws_sigv4",
  "parameters": {
    "service_name": "bedrock",
    "region": bedrock_region,
    "model_id": urllib.parse.quote_plus(deepseek_model_bedrock_arn),
    "temperature": 0,
    "max_gen_len": 4000
  },
  "actions": [
    {
      "action_type": "PREDICT",
      "method": "POST",
      "url": f"https://bedrock-runtime.{bedrock_region}.amazonaws.com/model/${{parameters.model_id}}/invoke",
      "headers": {
        "content-type": "application/json"
      },
      "request_body": "{ \"prompt\": \"<｜begin▁of▁sentence｜><｜User｜>${parameters.inputs}<｜Assistant｜>\", \"temperature\": ${parameters.temperature}, \"max_gen_len\": ${parameters.max_gen_len} }",
      "post_process_function": "\n      return '{' +\n               '\"name\": \"response\",'+\n               '\"dataAsMap\": {' +\n                  '\"completion\":\"' + escape(params.generation) + '\"}' +\n             '}';\n    "
    }
  ]
}

deepseek_connector_id = helper.create_connector_with_role(connector_role_inline_policy,
                                                 connector_role_name,
                                                 create_connector_role_name,
                                                 create_connector_input,
                                                 sleep_time_in_seconds=10)

Step1: Create IAM role configued in connector
Created role: my_test_role_deepseek_r1_bedrock
----------
Step 2: Configure IAM role in OpenSearch
Step 2.1: Create IAM role for Signing create connector request
Created role: my_test_role_deepseek_r1_bedrock_create
----------
Step 2.2: Map IAM role my_test_role_deepseek_r1_bedrock_create to OpenSearch permission role
{"status":"OK","message":"'ml_full_access' updated."}
----------
Step 3: Create connector in OpenSearch
{"connector_id":"quRGvpQBFSAM-WczVLIM"}
----------


In [16]:
model_name='DeepSeek R1 Model on Bedrock'
description='DeepSeek R1 Model on Bedrock'
deepseek_model_id = helper.create_model(model_name, description, deepseek_connector_id)
deepseek_model_id

{"task_id":"lilGvpQBts7fa6byWx0I","status":"CREATED","model_id":"lylGvpQBts7fa6byWx0x"}


'lylGvpQBts7fa6byWx0x'

In [17]:
request_data={
  "parameters": {
    "inputs": "hello"
  }
}
response = helper.predict(deepseek_model_id, request_data)
pretty_print_json(response)

{
    "inference_results": [
        {
            "output": [
                {
                    "dataAsMap": {
                        "completion": "<think>\n\n</think>\n\nHello! How can I assist you today? \ud83d\ude0a"
                    },
                    "name": "response"
                }
            ],
            "status_code": 200
        }
    ]
}


## 2.3 Use DeepSeek R1 model on Bedrock

Refer to this [tutorial](https://github.com/opensearch-project/ml-commons/blob/main/docs/tutorials/aws/RAG_with_DeepSeek_R1_model_on_Sagemaker.md) for more details.

Make sure you already have DeepSeek R1 model deployed to SageMaker before running next step.

In [18]:
sagemaker_inference_endpoint_arn = getpass("Enter your DeepSeek Model SageMaker Inference Endpoint ARN: ")
sagemaker_inference_endpoint_url = getpass("Enter your DeepSeek Model SageMaker Inference Endpoint URL: ")

Enter your DeepSeek Model SageMaker Inference Endpoint ARN:  ········
Enter your DeepSeek Model SageMaker Inference Endpoint URL:  ········


In [19]:
# You can use existing role if the role permission and trust relationship are correct.
# But highly suggest to specify new role names. AIConnectorHelper will create role automatically with correct permission.
# If you see permission issue, always try to create new roles.
connector_role_name = f'{connector_role_prefix}_deepseek_r1_sagemaker'
create_connector_role_name = f'{connector_role_prefix}_deepseek_r1_sagemaker_create'

sagemaker_endpoint_region = 'us-east-1' # SageMaker endpoint region could be different with OpenSearch domain region
connector_role_inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": [
                sagemaker_inference_endpoint_arn
            ]
        }
    ]
}

create_connector_input = {
  "name": "DeepSeek R1 model connector",
  "description": "Connector for my SageMaker DeepSeek model",
  "version": "1.0",
  "protocol": "aws_sigv4",
  "parameters": {
    "service_name": "sagemaker",
    "region": sagemaker_endpoint_region,
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.7,
    "max_new_tokens": 512
  },
  "actions": [
    {
      "action_type": "PREDICT",
      "method": "POST",
      "url": sagemaker_inference_endpoint_url,
      "headers": {
        "content-type": "application/json"
      },
      "request_body": "{ \"inputs\": \"${parameters.inputs}\", \"parameters\": {\"do_sample\": ${parameters.do_sample}, \"top_p\": ${parameters.top_p}, \"temperature\": ${parameters.temperature}, \"max_new_tokens\": ${parameters.max_new_tokens}} }",
      "post_process_function": "\n      if (params.result == null || params.result.length == 0) {\n        throw new Exception('No response available');\n      }\n      \n      def completion = params.result[0].generated_text;\n      return '{' +\n               '\"name\": \"response\",'+\n               '\"dataAsMap\": {' +\n                  '\"completion\":\"' + escape(completion) + '\"}' +\n             '}';\n    "
    }
  ]
}

deepseek_connector_id = helper.create_connector_with_role(connector_role_inline_policy,
                                                 connector_role_name,
                                                 create_connector_role_name,
                                                 create_connector_input,
                                                 sleep_time_in_seconds=10)

Step1: Create IAM role configued in connector
Created role: my_test_role_deepseek_r1_sagemaker
----------
Step 2: Configure IAM role in OpenSearch
Step 2.1: Create IAM role for Signing create connector request
Created role: my_test_role_deepseek_r1_sagemaker_create
----------
Step 2.2: Map IAM role my_test_role_deepseek_r1_sagemaker_create to OpenSearch permission role
{"status":"OK","message":"'ml_full_access' updated."}
----------
Step 3: Create connector in OpenSearch
{"connector_id":"mSlIvpQBts7fa6byGB24"}
----------


In [20]:
model_name='DeepSeek R1 Model on SageMaker'
description='DeepSeek R1 Model on SageMaker'
deepseek_model_id = helper.create_model(model_name, description, deepseek_connector_id)
deepseek_model_id

{"task_id":"milIvpQBts7fa6byJR0C","status":"CREATED","model_id":"mylIvpQBts7fa6byJR0c"}


'mylIvpQBts7fa6byJR0c'

In [21]:
request_data={
  "parameters": {
    "inputs": "hello"
  }
}
response = helper.predict(deepseek_model_id, request_data)
pretty_print_json(response)

{
    "inference_results": [
        {
            "output": [
                {
                    "dataAsMap": {
                        "completion": "hello<think>\n\n</think>\n\nHello! How can I assist you today? \ud83d\ude0a"
                    },
                    "name": "response"
                }
            ],
            "status_code": 200
        }
    ]
}


# 3 Vector Database

Refer to this [tutorial](https://opensearch.org/docs/latest/search-plugins/neural-search-tutorial/) for more details.

## 3.1 Create Bedrock Titan Embedding Model

In [22]:
# You can use existing role if the role permission and trust relationship are correct.
# But highly suggest to specify new role names. AIConnectorHelper will create role automatically with correct permission.
# If you see permission issue, always try to create new roles.
connector_role_name = f'{connector_role_prefix}_bedrock_titan_embedding_v2'
create_connector_role_name = f'{connector_role_prefix}_bedrock_titan_embedding_v2_create'

bedrock_region = 'us-east-1' # bedrock region could be different with OpenSearch domain region

connector_role_inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Effect": "Allow",
            "Resource": "arn:aws:bedrock:*::foundation-model/amazon.titan-embed-text-v2:0"
        }
    ]
}


create_connector_input = {
  "name": "Amazon Bedrock Connector: titan embedding v2",
  "description": "The connector to bedrock Titan embedding model",
  "version": 1,
  "protocol": "aws_sigv4",
  "parameters": {
    "region": bedrock_region,
    "service_name": "bedrock",
    "model": "amazon.titan-embed-text-v2:0",
    "dimensions": 1024,
    "normalize": True,
    "embeddingTypes": ["float"]
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
      "headers": {
        "content-type": "application/json",
        "x-amz-content-sha256": "required"
      },
      "request_body": "{ \"inputText\": \"${parameters.inputText}\", \"dimensions\": ${parameters.dimensions}, \"normalize\": ${parameters.normalize}, \"embeddingTypes\": ${parameters.embeddingTypes} }",
      "pre_process_function": "connector.pre_process.bedrock.embedding",
      "post_process_function": "connector.post_process.bedrock.embedding"
    }
  ]
}

embedding_connector_id = helper.create_connector_with_role(connector_role_inline_policy,
                                                 connector_role_name,
                                                 create_connector_role_name,
                                                 create_connector_input,
                                                 sleep_time_in_seconds=10)

Step1: Create IAM role configued in connector
Created role: my_test_role_bedrock_titan_embedding_v2
----------
Step 2: Configure IAM role in OpenSearch
Step 2.1: Create IAM role for Signing create connector request
Created role: my_test_role_bedrock_titan_embedding_v2_create
----------
Step 2.2: Map IAM role my_test_role_bedrock_titan_embedding_v2_create to OpenSearch permission role
{"status":"OK","message":"'ml_full_access' updated."}
----------
Step 3: Create connector in OpenSearch
{"connector_id":"q-RIvpQBFSAM-Wcz5bI5"}
----------


In [23]:
model_name='Bedrock Titan Embedding Model V2'
description='Bedrock Titan Embedding Model V2'
embedding_model_id = helper.create_model(model_name, description, embedding_connector_id)
embedding_model_id

{"task_id":"nSlIvpQBts7fa6by-x08","status":"CREATED","model_id":"nilIvpQBts7fa6by-x1p"}


'nilIvpQBts7fa6by-x1p'

In [24]:
request_data={
    "parameters": {
        "inputText": "hello"
    }
}
helper.predict(embedding_model_id, request_data)

'{"inference_results":[{"output":[{"name":"sentence_embedding","data_type":"FLOAT32","shape":[1024],"data":[-0.055150498,0.045168508,...]}],"status_code":200}]}'

## 3.2 Create Vector Database

In [25]:
# Create ingest pipeline
ingest_pipeline_name = 'text-embedding-ingest-pipeline'
ingest_pipeline_config = {
  "description": "Text embedding ingest pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": embedding_model_id,
        "field_map": {
          "text": "passage_embedding"
        }
      }
    }
  ]
}

helper.create_ingest_pipeline(ingest_pipeline_name, ingest_pipeline_config)

'{"acknowledged":true}'

In [26]:
# Create k-NN index
index_name = 'population_data'
index_mapping = {
  "settings": {
    "index.knn": True,
    "default_pipeline": ingest_pipeline_name
  },
  "mappings": {
    "properties": {
      "id": {
        "type": "text"
      },
      "passage_embedding": {
        "type": "knn_vector",
        "dimension": 1024,
        "method": {
          "engine": "lucene",
          "space_type": "l2",
          "name": "hnsw",
          "parameters": {}
        }
      },
      "text": {
        "type": "text"
      }
    }
  }
}
helper.create_index(index_name, index_mapping)

'{"acknowledged":true,"shards_acknowledged":true,"index":"population_data"}'

In [27]:
# Load data
population_data = [
    {"text": "Chart and table of population level and growth rate for the Ogden-Layton metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of Ogden-Layton in 2023 is 750,000, a 1.63% increase from 2022.\nThe metro area population of Ogden-Layton in 2022 was 738,000, a 1.79% increase from 2021.\nThe metro area population of Ogden-Layton in 2021 was 725,000, a 1.97% increase from 2020.\nThe metro area population of Ogden-Layton in 2020 was 711,000, a 2.16% increase from 2019."},
    {"text": "Chart and table of population level and growth rate for the New York City metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\\nThe current metro area population of New York City in 2023 is 18,937,000, a 0.37% increase from 2022.\\nThe metro area population of New York City in 2022 was 18,867,000, a 0.23% increase from 2021.\\nThe metro area population of New York City in 2021 was 18,823,000, a 0.1% increase from 2020.\\nThe metro area population of New York City in 2020 was 18,804,000, a 0.01% decline from 2019."},
    {"text": "Chart and table of population level and growth rate for the Chicago metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\\nThe current metro area population of Chicago in 2023 is 8,937,000, a 0.4% increase from 2022.\\nThe metro area population of Chicago in 2022 was 8,901,000, a 0.27% increase from 2021.\\nThe metro area population of Chicago in 2021 was 8,877,000, a 0.14% increase from 2020.\\nThe metro area population of Chicago in 2020 was 8,865,000, a 0.03% increase from 2019."},
    {"text": "Chart and table of population level and growth rate for the Miami metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\\nThe current metro area population of Miami in 2023 is 6,265,000, a 0.8% increase from 2022.\\nThe metro area population of Miami in 2022 was 6,215,000, a 0.78% increase from 2021.\\nThe metro area population of Miami in 2021 was 6,167,000, a 0.74% increase from 2020.\\nThe metro area population of Miami in 2020 was 6,122,000, a 0.71% increase from 2019."},
    {"text": "Chart and table of population level and growth rate for the Austin metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\\nThe current metro area population of Austin in 2023 is 2,228,000, a 2.39% increase from 2022.\\nThe metro area population of Austin in 2022 was 2,176,000, a 2.79% increase from 2021.\\nThe metro area population of Austin in 2021 was 2,117,000, a 3.12% increase from 2020.\\nThe metro area population of Austin in 2020 was 2,053,000, a 3.43% increase from 2019."},
    {"text": "Chart and table of population level and growth rate for the Seattle metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\\nThe current metro area population of Seattle in 2023 is 3,519,000, a 0.86% increase from 2022.\\nThe metro area population of Seattle in 2022 was 3,489,000, a 0.81% increase from 2021.\\nThe metro area population of Seattle in 2021 was 3,461,000, a 0.82% increase from 2020.\\nThe metro area population of Seattle in 2020 was 3,433,000, a 0.79% increase from 2019."},
]
helper.bulk_ingest(index_name, population_data)

Bulk insert successful
{
  "took": 25,
  "ingest_took": 203,
  "errors": false,
  "items": [
    {
      "index": {
        "_index": "population_data",
        "_id": "oClJvpQBts7fa6bynh3a",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 3,
          "successful": 3,
          "failed": 0
        },
        "_seq_no": 0,
        "_primary_term": 1,
        "status": 201
      }
    },
    {
      "index": {
        "_index": "population_data",
        "_id": "oSlJvpQBts7fa6bynh3a",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 3,
          "successful": 3,
          "failed": 0
        },
        "_seq_no": 1,
        "_primary_term": 1,
        "status": 201
      }
    },
    {
      "index": {
        "_index": "population_data",
        "_id": "oilJvpQBts7fa6bynh3a",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 3,
          "successful": 3,
    

# 4 Run retrieval-augmented generation (RAG)

In [28]:
# Configure search pipeline
rag_pipeline_name = 'rag-pipeline-deepseek'
rag_pipeline_config = {
  "response_processors": [
    {
      "retrieval_augmented_generation": {
        "tag": "Demo pipeline",
        "description": "Demo pipeline Using DeepSeek R1",
        "model_id": deepseek_model_id,
        "context_field_list": [
          "text"
        ],
        "system_prompt": "You are a helpful assistant.",
        "user_instructions": "Generate a concise and informative answer in less than 100 words for the given question"
      }
    }
  ]
}
helper.create_search_pipeline(rag_pipeline_name, rag_pipeline_config)

'{"acknowledged":true}'

In [29]:
question = "What's the population increase of New York City from 2021 to 2023? How is the trending comparing with Miami?"
query = {
  "query": {
    "neural": {
      "passage_embedding": {
        "query_text": question,
        "model_id": embedding_model_id,
        "k": 5
      }
    }
  },
  "size": 4,
  "_source": [
    "text"
  ],
  "ext": {
    "generative_qa_parameters": {
      "llm_model": "bedrock/claude",
      "llm_question": question,
      "context_size": 5,
      "timeout": 15
    }
  }
}
search_response = helper.search(index_name, query, rag_pipeline_name)
pretty_print_json(search_response)

{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 5,
        "total": 5
    },
    "ext": {
        "retrieval_augmented_generation": {
            "answer": "You are a helpful assistant.\\nGenerate a concise and informative answer in less than 100 words for the given question\\nSEARCH RESULT 1: Chart and table of population level and growth rate for the New York City metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\\nThe current metro area population of New York City in 2023 is 18,937,000, a 0.37% increase from 2022.\\nThe metro area population of New York City in 2022 was 18,867,000, a 0.23% increase from 2021.\\nThe metro area population of New York City in 2021 was 18,823,000, a 0.1% increase from 2020.\\nThe metro area population of New York City in 2020 was 18,804,000, a 0.01% decline from 2019.\\nSEARCH RESULT 2: Chart and table of population level and growth rate for the Miami metro ar